In [1]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile


/home/hemanth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hemanth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hemanth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hemanth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
from tensorflow.keras.callbacks import TensorBoard

In [3]:
NAME = "Plant_Classification-CNN"

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [4]:
k = os.listdir('/home/hemanth/Tensorflow-GPU/Plant_Dataset/MK/D2/train_patch')
for i in k:
    print(len(os.listdir('/home/hemanth/Tensorflow-GPU/Plant_Dataset/MK/D2/train_patch/{}'.format(i))))
    print(len(os.listdir('/home/hemanth/Tensorflow-GPU/Plant_Dataset/MK/D2/test_patch/{}'.format(i))))
    break

788
200


In [5]:
print(len(k))

44


In [6]:
import cv2

image = cv2.imread('/home/hemanth/Tensorflow-GPU/Plant_Dataset/MK/D2/train_patch/Class4/Class4(4)R180_00042.jpg')
image.shape

(256, 256, 3)

In [7]:
model = tf.keras.models.Sequential([
# YOUR CODE HERE
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(44,activation='softmax')
])

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 115200)            0         
__________

In [10]:
TRAINING_DIR = '/home/hemanth/Tensorflow-GPU/Plant_Dataset/MK/D2/train_patch/'
train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range=0.2,width_shift_range=0.2,height_shift_range=0.2,rotation_range=40,zoom_range=0.2,fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,batch_size=50,class_mode='categorical',target_size=(256,256))

VALIDATION_DIR = '/home/hemanth/Tensorflow-GPU/Plant_Dataset/MK/D2/test_patch/'
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,batch_size=50,class_mode='categorical',target_size=(256,256))



Found 34672 images belonging to 44 classes.
Found 8800 images belonging to 44 classes.


In [11]:
history = model.fit_generator(train_generator,
                              epochs=5,verbose=1,
                              validation_data=validation_generator,callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
694/694 [==============================] - 987s 1s/step - loss: 2.8022 - acc: 0.1939 - val_loss: 2.0438 - val_acc: 0.3808
Epoch 2/5
694/694 [==============================] - 367s 529ms/step - loss: 1.7171 - acc: 0.4519 - val_loss: 1.6549 - val_acc: 0.4508
Epoch 3/5
694/694 [==============================] - 373s 538ms/step - loss: 1.2271 - acc: 0.5900 - val_loss: 1.1894 - val_acc: 0.6009
Epoch 4/5
694/694 [==============================] - 370s 533ms/step - loss: 1.0249 - acc: 0.6595 - val_loss: 1.0365 - val_acc: 0.6782
Epoch 5/5
694/694 [==============================] - 373s 538ms/step - loss: 0.8690 - acc: 0.7101 - val_loss: 0.7162 - val_acc: 0.7576


In [12]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

SyntaxError: unexpected EOF while parsing (<ipython-input-12-e9ed6b1008e8>, line 6)

In [13]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
